In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
sys.path.append("/home/files/feature_orthogonality")
from utils.session_config import setup_gpus

In [2]:
import tensorflow as tf
keras = tf.keras
import cv2
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
setup_gpus(memory_fraction=0.2)

1 Physical GPUs, 1 Logical GPUs


In [4]:
from tensorflow.keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, ReLU, BatchNormalization, GlobalAveragePooling2D
from utils.losses import L2_Orthogonal
from utils.BEP import convert_to_separate_perceptrons, Back_etching_propagation
from utils.layers import Etching_Layer

In [5]:
trained_model = tf.keras.models.load_model("mnist-3-cat-classifier.h5")

In [23]:
tf.keras.layers.GlobalAvgPool2D()(temp_model(x))[:,5]

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 0.00242411, -0.00173984, -0.00277075, -0.00349931, -0.01144431,
       -0.00793975, -0.01211818,  0.0085511 , -0.00830549, -0.00039793,
       -0.00319792, -0.0002434 , -0.00393798,  0.004533  , -0.00661029,
        0.00820457, -0.00904269, -0.01188685,  0.00609089, -0.0037124 ,
        0.00200081, -0.00265654, -0.00489497,  0.01595178,  0.00597185,
       -0.00370953,  0.01597578,  0.00600257,  0.00953018, -0.00087979,
        0.01679454, -0.00941896], dtype=float32)>

In [ ]:
bep_algorithm = Back_etching_propagation(trained_model)

In [ ]:
model = bep_algorithm.return_model()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
bep_algorithm.etching_layer_indice

# Etching

In [ ]:
#model.summary()

etching_idx = 17

model.layers[etching_idx].trainable = True
i = model.layers[etching_idx].output
_,o = model.layers[etching_idx+1](i)
etching_model = tf.keras.Model(model.input, o[:,0])

In [7]:
data_0 = np.load("mnist/0.npy")[...,np.newaxis]/255.
train_x = tf.data.Dataset.from_tensor_slices(data_0)
train_x = train_x.batch(32)

for x in train_x.take(1):
    print(x.shape)

(32, 28, 28, 1)


In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-4)
for x in train_x:
    with tf.GradientTape() as tape:
        pred = etching_model(x, training=True)
        loss = tf.reduce_mean(pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
model.layers[etching_idx].get_weights()

In [ ]:
bep_algorithm.etching_layer_indice

In [ ]:
for layer in model.layers:
    layer.trainable = False
etching_idx = 12
model.layers[etching_idx].trainable = True

In [ ]:
i = model.layers[etching_idx].output
_,o = model.layers[etching_idx+1](i)
print(o)
o = tf.gather(o, [2,3,4], axis=-1)
print(o)
etching_model = tf.keras.Model(model.input, o)
#i = model.layers[etching_idx].output
#o = model.layers[etching_idx+1](i)
#etching_model = tf.keras.Model(model.input, o[:,0])


# i = model.layers[etching_idx].output
# _, o = model.layers[etching_idx+1](i)
# etching_model = tf.keras.Model(model.input, o[0])

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1E-4)
for x in train_x:
    with tf.GradientTape() as tape:
        pred = etching_model(x, training=True)
        loss = tf.reduce_mean(pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [ ]:
model.layers[etching_idx].get_weights()[0][:,3]

In [ ]:
#etching_model.summary().
etching_model.layers[-3].get_weights()

In [ ]:
etching_model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False
model.layers[-2].trainable = True

In [ ]:
model.input

In [ ]:
for x in train_x.take(1):
    _, preds = model(x)

In [ ]:
tf.reduce_mean(preds[-1][0])

In [ ]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()
for x in train_x:
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))